In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from timeit import default_timer as timer
# import time

from visualizations import *

In [ ]:
# YOLO MODEL

In [ ]:
class yolo_tf:
    w_img = 1280
    h_img = 720

    weights_file = 'weights/YOLO_small.ckpt'
    alpha = 0.1
    threshold = 0.3
    iou_threshold = 0.5

    result_list = None
    classes =  ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair",
                "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant",
                "sheep", "sofa", "train","tvmonitor"]

    def __init__(self):
        self.build_networks()

        
    def build_networks(self):
        print("Building YOLO_small ...")
        self.x = tf.placeholder('float32',[None,448,448,3])
        # self.x = tf.placeholder('float32',[None,252, 1280, 3])
        
        # Adding layers
        self.conv_1 = self.conv_layer(1,self.x,64,7,2)
        self.pool_2 = self.pooling_layer(2,self.conv_1,2,2)
        self.conv_3 = self.conv_layer(3,self.pool_2,192,3,1)
        self.pool_4 = self.pooling_layer(4,self.conv_3,2,2)
        self.conv_5 = self.conv_layer(5,self.pool_4,128,1,1)
        self.conv_6 = self.conv_layer(6,self.conv_5,256,3,1)
        self.conv_7 = self.conv_layer(7,self.conv_6,256,1,1)
        self.conv_8 = self.conv_layer(8,self.conv_7,512,3,1)
        self.pool_9 = self.pooling_layer(9,self.conv_8,2,2)
        self.conv_10 = self.conv_layer(10,self.pool_9,256,1,1)
        self.conv_11 = self.conv_layer(11,self.conv_10,512,3,1)
        self.conv_12 = self.conv_layer(12,self.conv_11,256,1,1)
        self.conv_13 = self.conv_layer(13,self.conv_12,512,3,1)
        self.conv_14 = self.conv_layer(14,self.conv_13,256,1,1)
        self.conv_15 = self.conv_layer(15,self.conv_14,512,3,1)
        self.conv_16 = self.conv_layer(16,self.conv_15,256,1,1)
        self.conv_17 = self.conv_layer(17,self.conv_16,512,3,1)
        self.conv_18 = self.conv_layer(18,self.conv_17,512,1,1)
        self.conv_19 = self.conv_layer(19,self.conv_18,1024,3,1)
        self.pool_20 = self.pooling_layer(20,self.conv_19,2,2)
        self.conv_21 = self.conv_layer(21,self.pool_20,512,1,1)
        self.conv_22 = self.conv_layer(22,self.conv_21,1024,3,1)
        self.conv_23 = self.conv_layer(23,self.conv_22,512,1,1)
        self.conv_24 = self.conv_layer(24,self.conv_23,1024,3,1)
        self.conv_25 = self.conv_layer(25,self.conv_24,1024,3,1)
        self.conv_26 = self.conv_layer(26,self.conv_25,1024,3,2)
        self.conv_27 = self.conv_layer(27,self.conv_26,1024,3,1)
        self.conv_28 = self.conv_layer(28,self.conv_27,1024,3,1)
        
        # Adding fully connected layers
        self.fc_29 = self.fc_layer(29,self.conv_28,512,flat=True,linear=False)
        self.fc_30 = self.fc_layer(30,self.fc_29,4096,flat=False,linear=False)
        
        #skipped dropout_31
        self.fc_32 = self.fc_layer(32, self.fc_30, 1470, flat=False, linear=True)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        self.saver.restore(self.sess, self.weights_file)
        
        print("Loading complete!")
        

    def conv_layer(self,idx,inputs,filters,size,stride):
        channels = inputs.get_shape()[3]
        weight = tf.Variable(tf.truncated_normal([size,size,int(channels),filters], stddev=0.1))
        biases = tf.Variable(tf.constant(0.1, shape=[filters]))

        pad_size = size//2
        pad_mat = np.array([[0,0],[pad_size,pad_size],[pad_size,pad_size],[0,0]])
        inputs_pad = tf.pad(inputs,pad_mat)

        conv = tf.nn.conv2d(inputs_pad, weight, strides=[1, stride, stride, 1], padding='VALID',name=str(idx)+'_conv')
        conv_biased = tf.add(conv,biases,name=str(idx)+'_conv_biased')
        print('Layer  %d : Type = Conv, Size = %d * %d, Stride = %d, Filters = %d, Input channels = %d' % (idx,size,size,stride,filters,int(channels)))
        return tf.maximum(self.alpha*conv_biased,conv_biased,name=str(idx)+'_leaky_relu')

    
    def pooling_layer(self,idx,inputs,size,stride):
        print ('Layer  %d : Type = Pool, Size = %d * %d, Stride = %d' % (idx,size,size,stride))
        return tf.nn.max_pool(inputs, ksize=[1, size, size, 1],strides=[1, stride, stride, 1], padding='SAME',name=str(idx)+'_pool')

    
    def fc_layer(self,idx,inputs,hiddens,flat = False,linear = False):
        input_shape = inputs.get_shape().as_list()
        if flat:
            dim = input_shape[1]*input_shape[2]*input_shape[3]
            inputs_transposed = tf.transpose(inputs,(0,3,1,2))
            inputs_processed = tf.reshape(inputs_transposed, [-1,dim])
        else:
            dim = input_shape[1]
            inputs_processed = inputs
        weight = tf.Variable(tf.truncated_normal([dim,hiddens], stddev=0.1))
        biases = tf.Variable(tf.constant(0.1, shape=[hiddens]))
        print ('Layer  %d : Type = Full, Hidden = %d, Input dimension = %d, Flat = %d, Activation = %d' % (idx,hiddens,int(dim),int(flat),1-int(linear))	)
        if linear : return tf.add(tf.matmul(inputs_processed,weight),biases,name=str(idx)+'_fc')
        ip = tf.add(tf.matmul(inputs_processed,weight),biases)
        return tf.maximum(self.alpha*ip,ip,name=str(idx)+'_fc')


In [ ]:
# FNS FOR DETECTION

In [ ]:
def detect_from_cvmat(yolo,img):
    yolo.h_img,yolo.w_img,_ = img.shape
    img_resized = cv2.resize(img, (448, 448))
    img_resized_np = np.asarray( img_resized )
    inputs = np.zeros((1,448,448,3),dtype='float32')
    inputs[0] = (img_resized_np/255.0)*2.0-1.0
    in_dict = {yolo.x: inputs}
    net_output = yolo.sess.run(yolo.fc_32,feed_dict=in_dict)
    result = interpret_output(yolo, net_output[0])
    yolo.result_list = result

In [ ]:
def detect_from_file(yolo,filename):
    detect_from_cvmat(yolo, filename)


# fn to interpret model outputs/ predictions
def interpret_output(yolo,output):
    probs = np.zeros((7,7,2,20))
    class_probs = np.reshape(output[0:980],(7,7,20))
    scales = np.reshape(output[980:1078],(7,7,2))
    boxes = np.reshape(output[1078:],(7,7,2,4))
    offset = np.transpose(np.reshape(np.array([np.arange(7)]*14),(2,7,7)),(1,2,0))

    boxes[:,:,:,0] += offset
    boxes[:,:,:,1] += np.transpose(offset,(1,0,2))
    boxes[:,:,:,0:2] = boxes[:,:,:,0:2] / 7.0
    boxes[:,:,:,2] = np.multiply(boxes[:,:,:,2],boxes[:,:,:,2])
    boxes[:,:,:,3] = np.multiply(boxes[:,:,:,3],boxes[:,:,:,3])

    boxes[:,:,:,0] *= yolo.w_img
    boxes[:,:,:,1] *= yolo.h_img
    boxes[:,:,:,2] *= yolo.w_img
    boxes[:,:,:,3] *= yolo.h_img

    for i in range(2):
        for j in range(20):
            probs[:,:,i,j] = np.multiply(class_probs[:,:,j],scales[:,:,i])

    filter_mat_probs = np.array(probs>=yolo.threshold,dtype='bool')
    filter_mat_boxes = np.nonzero(filter_mat_probs)
    boxes_filtered = boxes[filter_mat_boxes[0],filter_mat_boxes[1],filter_mat_boxes[2]]
    probs_filtered = probs[filter_mat_probs]
    classes_num_filtered = np.argmax(filter_mat_probs,axis=3)[filter_mat_boxes[0],filter_mat_boxes[1],filter_mat_boxes[2]]

    argsort = np.array(np.argsort(probs_filtered))[::-1]
    boxes_filtered = boxes_filtered[argsort]
    probs_filtered = probs_filtered[argsort]
    classes_num_filtered = classes_num_filtered[argsort]

    for i in range(len(boxes_filtered)):
        if probs_filtered[i] == 0 : continue
        for j in range(i+1,len(boxes_filtered)):
            if iou(boxes_filtered[i],boxes_filtered[j]) > yolo.iou_threshold :
                probs_filtered[j] = 0.0

    filter_iou = np.array(probs_filtered>0.0,dtype='bool')
    boxes_filtered = boxes_filtered[filter_iou]
    probs_filtered = probs_filtered[filter_iou]
    classes_num_filtered = classes_num_filtered[filter_iou]

    result = []
    for i in range(len(boxes_filtered)):
        result.append([yolo.classes[classes_num_filtered[i]],boxes_filtered[i][0],boxes_filtered[i][1],boxes_filtered[i][2],boxes_filtered[i][3],probs_filtered[i]])

    return result

In [ ]:
# fn to draw bounding boxes and results

In [ ]:
def draw_results(img, yolo, fps):
    img_cp = img.copy()
    results = yolo.result_list

    # draw the highlighted background
    # img_cp = draw_background_highlight(img_cp, image_lane, yolo.w_img)

    window_list = []
    for i in range(len(results)):
        x = int(results[i][1])
        y = int(results[i][2])
        w = int(results[i][3])//2
        h = int(results[i][4])//2
        cv2.rectangle(img_cp,(x-w,y-h),(x+w,y+h),(0,0,255),4)
        cv2.rectangle(img_cp,(x-w,y-h-20),(x+w,y-h),(125,125,125),-1)
        # cv2.putText(img_cp,results[i][0] + ' : %.2f' % results[i][5],(x-w+5,y-h-7),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0),1)
        cv2.putText(img_cp,results[i][0],(x-w+5,y-h-7),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0),1)
        if results[i][0] == "car" or results[i][0] == "bus":
            window_list.append(((x-w,y-h),(x+w,y+h)))

    # draw vehicle thumbnails
    draw_thumbnails(img_cp, img, window_list)

    # draw speed
    draw_speed(img_cp, fps, yolo.w_img)

    # draw lane status
    # draw_lane_status(img_cp,lane_info)

    return img_cp


def iou(box1,box2):
    tb = min(box1[0]+0.5*box1[2],box2[0]+0.5*box2[2])-max(box1[0]-0.5*box1[2],box2[0]-0.5*box2[2])
    lr = min(box1[1]+0.5*box1[3],box2[1]+0.5*box2[3])-max(box1[1]-0.5*box1[3],box2[1]-0.5*box2[3])
    if tb < 0 or lr < 0 : intersection = 0
    else : intersection =  tb*lr
    return intersection / (box1[2]*box1[3] + box2[2]*box2[3] - intersection)


In [ ]:
# Instantiating Model from Class

In [ ]:
yolo = yolo_tf()

def vehicle_detection_yolo(image):
    start = timer()
    detect_from_file(yolo, image)

    # computing frame per second
    fps = 1.0 / (timer() - start)
    
    # drawing visualizations on frame
    yolo_result = draw_results(image, yolo, fps)

    return yolo_result

In [ ]:
# YOLO model created

In [ ]:
# LANE DETECTION 

In [ ]:
def lane_detect(im):
    
    # gray scale conversion
    grayIm = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    # smoothening
    kernel_size = 9; #bigger kernel-->more smoothing
    smoothedIm = cv2.GaussianBlur(grayIm, (kernel_size, kernel_size), 0)
    
    minVal = 60
    maxVal = 150
    edgesIm = cv2.Canny(smoothedIm, minVal, maxVal)
    vertices = np.array([[(0,im.shape[0]),(465, 320), (475, 320), (im.shape[1],im.shape[0])]], dtype=np.int32)

    # defining a blank mask to start with, 0s with shape same as that of edgesIm
    mask = np.zeros_like(edgesIm)   
          
    # filling pixels inside the polygon defined by vertices with the fill color  
    color = 255
    cv2.fillPoly(mask, vertices, color)
    
    maskedIm = cv2.bitwise_and(edgesIm, mask)
    maskedIm3Channel = cv2.cvtColor(maskedIm, cv2.COLOR_GRAY2BGR)

    # Hough-Lines Transform
    rho = 2 #distance resolution in pixels of the Hough grid
    theta = np.pi/180    # ngular resolution in radians of the Hough grid
    threshold = 45    #minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40    #minimum number of pixels making up a line
    max_line_gap = 100    #maximum gap in pixels between connectable line segments
    lines = cv2.HoughLinesP(maskedIm, rho, theta, threshold, np.array([]), 
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    
    if lines is not None and len(lines) > 2:
        
        # Draw all lines onto image
        allLines = np.zeros_like(maskedIm)
        for i in range(len(lines)):
            for x1,y1,x2,y2 in lines[i]:
                cv2.line(allLines,(x1,y1),(x2,y2),(255,255,0),2) # plot line

        # Plot all lines found
        plt.figure(7)
        plt.imshow(allLines,cmap='gray')
        plt.title('All Hough Lines Found')

        # Separating Lines acc to Positive/Negative Slope
        
        # 1 Separating line segments by their slope to decide left line vs. the right line
        slopePositiveLines = [] # x1 y1 x2 y2 slope
        slopeNegativeLines = []
        yValues = []

        # 2 Looping through all lines
        addedPos = False
        addedNeg = False
        for currentLine in lines:   
            # Get points of current Line
            for x1,y1,x2,y2 in currentLine:
                lineLength = ((x2-x1)**2 + (y2-y1)**2)**.5 # get line length
                if lineLength > 30: # if line is long enough
                    if x2 != x1: # dont divide by zero
                        slope = (y2-y1)/(x2-x1) # get slope line
                        if slope > 0: 
                            # Check angle of line w/ xaxis. dont want vertical/horizontal lines
                            tanTheta = np.tan((abs(y2-y1))/(abs(x2-x1))) # tan(theta) value
                            ang = np.arctan(tanTheta)*180/np.pi
                            if abs(ang) < 85 and abs(ang) > 20:
                                slopeNegativeLines.append([x1,y1,x2,y2,-slope]) # add positive slope line
                                yValues.append(y1)
                                yValues.append(y2)
                                addedPos = True # note that we added a positive slope line
                        if slope < 0:
                            # Check angle of line w/ xaxis. dont want vertical/horizontal lines
                            tanTheta = np.tan((abs(y2-y1))/(abs(x2-x1))) # tan(theta) value
                            ang = np.arctan(tanTheta)*180/np.pi
                            if abs(ang) < 85 and abs(ang) > 20:
                                slopePositiveLines.append([x1,y1,x2,y2,-slope]) # add negative slope line
                                yValues.append(y1)
                                yValues.append(y2)
                                addedNeg = True # note that we added a negative slope line


        # 3 If we didn't get any positive lines, go through again and just add any positive slope lines         
        if not addedPos:
            for currentLine in lines:
                for x1,y1,x2,y2 in currentLine:
                    slope = (y2-y1)/(x2-x1)
                    if slope > 0:
                        # Check angle of line w/ xaxis. dont want vertical/horizontal lines
                        tanTheta = np.tan((abs(y2-y1))/(abs(x2-x1))) # tan(theta) value
                        ang = np.arctan(tanTheta)*180/np.pi
                        if abs(ang) < 80 and abs(ang) > 15:
                            slopeNegativeLines.append([x1,y1,x2,y2,-slope])
                            yValues.append(y1)
                            yValues.append(y2)

        # 4 If we didn't get any negative lines, go through again and just add any negative slope lines
        if not addedNeg:
            for currentLine in lines:
                for x1,y1,x2,y2 in currentLine:
                    slope = (y2-y1)/(x2-x1)
                    if slope < 0:
                        # Check angle of line w/ xaxis. dont want vertical/horizontal lines
                        tanTheta = np.tan((abs(y2-y1))/(abs(x2-x1))) # tan(theta) value
                        ang = np.arctan(tanTheta)*180/np.pi
                        if abs(ang) < 85 and abs(ang) > 15:
                            slopePositiveLines.append([x1,y1,x2,y2,-slope])           
                            yValues.append(y1)
                            yValues.append(y2)


        if not addedPos or not addedNeg:
            print('Not enough lines found')


        # Getting Positive/Negative Slope Averages
        
        # Average position of lines and extrapolate to the top and bottom of the lane
        positiveSlopes = np.asarray(slopePositiveLines)[:,4]
        posSlopeMedian = np.median(positiveSlopes)
        posSlopeStdDev = np.std(positiveSlopes)
        posSlopesGood = []
        for slope in positiveSlopes:
           # if abs(slope-posSlopeMedian) < .9:
           if abs(slope-posSlopeMedian) < posSlopeMedian*.2:
                posSlopesGood.append(slope)
        posSlopeMean = np.mean(np.asarray(posSlopesGood))


        negativeSlopes = np.asarray(slopeNegativeLines)[:,4]
        negSlopeMedian = np.median(negativeSlopes)
        negSlopeStdDev = np.std(negativeSlopes)
        negSlopesGood = []
        for slope in negativeSlopes:
            if abs(slope-negSlopeMedian) < .9:
                negSlopesGood.append(slope)
        negSlopeMean = np.mean(np.asarray(negSlopesGood))

        
        # getting Average x coord when y coord Of Line = 0
        
        # 1 Positive Lines
        xInterceptPos = []
        for line in slopePositiveLines:
                x1 = line[0]
                y1 = im.shape[0]-line[1] # y axis is flipped
                slope = line[4]
                yIntercept = y1-slope*x1
                xIntercept = -yIntercept/slope # find x intercept based off y = mx+b
                if xIntercept == xIntercept: # checks for nan
                    xInterceptPos.append(xIntercept) # add x intercept

        xIntPosMed = np.median(xInterceptPos) # get median 
        xIntPosGood = [] # if not near median we get rid of that x point
        for line in slopePositiveLines:
                x1 = line[0]
                y1 = im.shape[0]-line[1]
                slope = line[4]
                yIntercept = y1-slope*x1
                xIntercept = -yIntercept/slope
                if abs(xIntercept-xIntPosMed) < .35*xIntPosMed: # check if near median
                    xIntPosGood.append(xIntercept)

        xInterceptPosMean = np.mean(np.asarray(xIntPosGood)) # average of good x intercepts for positive line

        # 2 Negative Lines 
        xInterceptNeg = []
        for line in slopeNegativeLines:
            x1 = line[0]
            y1 = im.shape[0]-line[1]
            slope = line[4]
            yIntercept = y1-slope*x1
            xIntercept = -yIntercept/slope
            if xIntercept == xIntercept: # check for nan
                    xInterceptNeg.append(xIntercept)

        xIntNegMed = np.median(xInterceptNeg)
        xIntNegGood = []
        for line in slopeNegativeLines:
            x1 = line[0]
            y1 = im.shape[0]-line[1]
            slope = line[4]
            yIntercept = y1-slope*x1
            xIntercept = -yIntercept/slope
            if abs(xIntercept-xIntNegMed)< .35*xIntNegMed: 
                    xIntNegGood.append(xIntercept)

        xInterceptNegMean = np.mean(np.asarray(xIntNegGood))

        
    
    # making new black image
    laneLines = np.zeros_like(edgesIm)   
    colorLines = im.copy()

    # Positive Slope Line
    slope = posSlopeMean
    x1 = xInterceptPosMean
    y1 = 0
    y2 = im.shape[0] - (im.shape[0]-im.shape[0]*.35)
    x2 = (y2-y1)/slope + x1

    # Plotting positive slope line
    x1 = int(round(x1))
    x2 = int(round(x2))
    y1 = int(round(y1))
    y2 = int(round(y2))
    cv2.line(laneLines,(x1,im.shape[0]-y1),(x2,im.shape[0]-y2),(255,255,0),2) # plot line
    cv2.line(colorLines,(x1,im.shape[0]-y1),(x2,im.shape[0]-y2),(0,255,0),4) # plot line on color image

    # Negative Slope Line
    slope = negSlopeMean
    x1N = xInterceptNegMean
    y1N = 0
    x2N = (y2-y1N)/slope + x1N

    # Plotting negative Slope Line
    x1N = int(round(x1N))
    x2N = int(round(x2N))
    y1N = int(round(y1N))
    cv2.line(laneLines,(x1N,im.shape[0]-y1N),(x2N,im.shape[0]-y2),(255,255,0),2)
    cv2.line(colorLines,(x1N,im.shape[0]-y1N),(x2N,im.shape[0]-y2),(0,255,0),4) # plot line on color iamge
    
    laneFill = im.copy()
    vertices = np.array([[(x1,im.shape[0]-y1),(x2,im.shape[0]-y2),  (x2N,im.shape[0]-y2),
                                          (x1N,im.shape[0]-y1N)]], dtype=np.int32)
    color = [241,255,1]
    cv2.fillPoly(laneFill, vertices, color)
    opacity = .25
    blendedIm =cv2.addWeighted(laneFill,opacity,im,1-opacity,0,im)
    cv2.line(blendedIm,(x1,im.shape[0]-y1),(x2,im.shape[0]-y2),(0,255,0),4) # plot line on color image
    cv2.line(blendedIm,(x1N,im.shape[0]-y1N),(x2N,im.shape[0]-y2),(0,255,0),4) # plot line on color image
    b,g,r = cv2.split(blendedIm) # get bgr channels
    outputIm = cv2.merge((r,g,b)) # make rgb


    return blendedIm


In [ ]:
# FINAL PIPELINE

In [ ]:
def proj_pipeline(img):

    im = vehicle_detection_yolo(img) #output from vehicle detection 
    output = lane_detect(im) # final output after lane detection 
    return output

In [ ]:
# TESTING ON SAMPLE INPUT VIDEO AND SAVING OUTPUT VIDEO FILE

In [ ]:
from moviepy.editor import VideoFileClip

In [ ]:
# give output video file path here
video_output = 'output_subclip.mp4'

# give input video file path here
# clip1 = VideoFileClip("project_input.mp4").subclip(0,20)
clip1 = VideoFileClip("project_input.mp4").subclip(25,40)

clip = clip1.fl_image(proj_pipeline)
clip.write_videofile(video_output, audio=False)

In [ ]:
# END